In [2]:
import numpy as np
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import (
    Flatten,
    Dense,
    AveragePooling2D,
    Dropout
)
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing import image
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    LearningRateScheduler
)

In [3]:
SHAPE = 224
BATCH_SIZE = 8
model = Xception(
    input_shape=(SHAPE, SHAPE, 3),
    include_top=False,
    weights='imagenet'
)

x = model.output
x = AveragePooling2D(pool_size=(2, 2))(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.1)(x)
x = Flatten()(x)
x = Dense(4, activation='softmax',
          kernel_regularizer=l2(.0005))(x)
model = Model(inputs=model.inputs, outputs=x)
opt = SGD(lr=0.0001, momentum=.9)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
valid_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    horizontal_flip=False
)
train_generator = train_datagen.flow_from_directory(
    'train/',
    target_size=(SHAPE, SHAPE),
    shuffle=True,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
)
valid_generator = valid_datagen.flow_from_directory(
    'valid/',
    target_size=(SHAPE, SHAPE),
    shuffle=True,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
)

Found 1056 images belonging to 4 classes.
Found 219 images belonging to 4 classes.


In [5]:
earlystop = EarlyStopping(monitor='val_loss',
                          patience=4,
                          verbose=1)
checkpoint = ModelCheckpoint(
    "model-weights/xception_checkpoint.h5",
    monitor="val_loss",
    mode="min",
    save_best_only=True,
    verbose=1
)

In [6]:
history = model.fit_generator(
    train_generator,
    epochs=20,
    callbacks=[earlystop, checkpoint],
    validation_data=valid_generator
)
# Save our model for inference
model.save("model-weights/xception.h5")

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
132/132 [==============================] - ETA: 0s - loss: 1.4128 - accuracy: 0.2472
Epoch 00001: val_loss improved from inf to 1.40804, saving model to model-weights\xception_checkpoint.h5
132/132 [==============================] - 596s 5s/step - loss: 1.4128 - accuracy: 0.2472 - val_loss: 1.4080 - val_accuracy: 0.2694
Epoch 2/20
132/132 [==============================] - ETA: 0s - loss: 1.3345 - accuracy: 0.3333
Epoch 00002: val_loss improved from 1.40804 to 1.33115, saving model to model-weights\xception_checkpoint.h5
132/132 [==============================] - 613s 5s/step - loss: 1.3345 - accuracy: 0.3333 - val_loss: 1.3311 - val_accuracy: 0.3607
Epoch 3/20
132/132 [==============================] - ETA: 0s - loss: 1.2798 - accuracy: 0.4195
Epoch 00003: val_loss improved from 1.33115 to 1.26672, saving model to model-weights\xception_checkpoint.h5
132/132 [==============================] - 637s 5

In [16]:
from tensorflow.keras.models import load_model
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

In [50]:
MODEL_PATH = 'model-weights/xception.h5'
model = load_model(MODEL_PATH)

In [51]:
def load_img(input_image, shape):
    img = Image.open(input_image).convert('RGB')
    img = img.resize((shape, shape))
    img = image.img_to_array(img)
    return np.reshape(img, [1, shape, shape, 3])/255

In [66]:
# pred_img = load_img("./train/cartier/cartier-0-5331.jpg", 224)
pred_img = load_img("omega.jpg", 224)

In [39]:
CLASS_DICT = {
    0: 'Cartier',
    1: 'Omega',
    2: 'Rolex',
    3: 'Seiko'
}

In [67]:
[pred for pred in model.predict(pred_img)[0]][0]

0.0048708576

In [40]:
pred = CLASS_DICT[np.argmax(model.predict(pred_img))]

In [41]:
pred

'Omega'

In [48]:
list(zip(list(CLASS_DICT.values()), [0.1, 0.1, 0.4, 0.4]))

[('Cartier', 0.1), ('Omega', 0.1), ('Rolex', 0.4), ('Seiko', 0.4)]

In [49]:
pred

'Omega'

In [64]:
list(zip(
			list(CLASS_DICT.values()),
			[prediction for prediction in pred]
		))

[('Cartier', 'O'), ('Omega', 'm'), ('Rolex', 'e'), ('Seiko', 'g')]

In [65]:
[prediction for prediction in pred]

['O', 'm', 'e', 'g', 'a']

In [73]:
pred = model.predict(pred_img)
type([round(prediction, 4) for prediction in map(float, pred[0])][0])

float